In [1]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
import requests
from github import Github
from git import Repo
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp
import configparser
from pprint import pprint
from datetime import datetime
#from gensim.models import Word2Vec

In [2]:
config = configparser.ConfigParser()
config.read('authentication.ini')
print(config.sections())
github_oauth = config['github']['api_key']

['github']


In [12]:
project_releases = pd.read_csv('volatile_projects_complete_links_limit10_filtered.csv')
project_releases.head()

,project_name,project_link,version_name,commit,timestamp
0,Dbeaver,https://github.com/dbeaver/dbeaver,21.1.4,"Commit(sha=""113a0a672f277a6e8181757a0c54f92d42...",29/7/2021 11:08
1,Dbeaver,https://github.com/dbeaver/dbeaver,21.1.3,"Commit(sha=""4430459a3fe06c6140aa40b71ddc41ddf8...",15/7/2021 8:06
2,Dbeaver,https://github.com/dbeaver/dbeaver,21.1.2,"Commit(sha=""b0693d44048a9c50e750b6df69cfe83fcb...",2/7/2021 13:34
3,Dbeaver,https://github.com/dbeaver/dbeaver,21.1.1,"Commit(sha=""073dfc26c7a065f5d5abf18be8cce8258a...",18/6/2021 13:50
4,Dbeaver,https://github.com/dbeaver/dbeaver,21.1.0,"Commit(sha=""17ce2d14317b1160ec9480da549028d182...",28/5/2021 5:16


In [6]:
project_refactoring_data = pd.read_csv('Effort_Estimation_Results_v2/Camel.csv')
project_refactoring_data.head()

,file,class,type,cbo,wmc,dit,rfc,lcom,totalMethods,staticMethods,...,anonymousClassesQty,subClassesQty,lambdasQty,uniqueWordsQty,modifiers,latest_commit,commit_compared_with,num_dependency,num_line_affected,actual_num_of_classes_touched
0,E:\SoftwareRemodularization\raw_sourcecode\Cam...,org.apache.camel.impl.cluster.ClusteredRouteCo...,class,16.0,23.0,4.0,37.0,85.0,19.0,0.0,...,0.0,1.0,1.0,83.0,1.0,1541a742d1fa2633ee2251eeea72904f3447c8f5,43d3c32077b7e8bbd04a46d28ccf128483ca8e57,18.0,215,1
1,E:\SoftwareRemodularization\raw_sourcecode\Cam...,org.apache.camel.model.DataFormatDefinitionAware,interface,1.0,2.0,1.0,0.0,1.0,2.0,0.0,...,0.0,0.0,0.0,18.0,1.0,1541a742d1fa2633ee2251eeea72904f3447c8f5,9c879480fe85d727a617fc4370e028390ffda68c,20.0,13,9
2,E:\SoftwareRemodularization\raw_sourcecode\Cam...,org.apache.camel.component.jsonata.JsonataEndp...,class,13.0,12.0,8.0,19.0,32.0,9.0,0.0,...,0.0,0.0,0.0,54.0,1.0,1541a742d1fa2633ee2251eeea72904f3447c8f5,ab201e6633e809b642e59df04f1630a5b94ed7c7,24.0,86,2
3,E:\SoftwareRemodularization\raw_sourcecode\Cam...,org.apache.camel.impl.health.DefaultHealthChec...,class,20.0,12.0,2.0,34.0,10.0,5.0,0.0,...,1.0,1.0,0.0,44.0,1.0,1541a742d1fa2633ee2251eeea72904f3447c8f5,0203888f910ce58f835c48e07502837a2a0c7805,16.0,187,1
4,E:\SoftwareRemodularization\raw_sourcecode\Cam...,org.apache.camel.impl.health.HealthCheckTest,class,5.0,6.0,2.0,20.0,6.0,4.0,0.0,...,0.0,1.0,0.0,33.0,1.0,1541a742d1fa2633ee2251eeea72904f3447c8f5,0203888f910ce58f835c48e07502837a2a0c7805,6.0,140,2


In [7]:
unique_releases_commit = pd.unique(project_refactoring_data['latest_commit'])
unique_commit = pd.unique(project_refactoring_data['commit_compared_with'])

In [8]:
print(unique_commit)
print(unique_releases_commit)

['43d3c32077b7e8bbd04a46d28ccf128483ca8e57'
 '9c879480fe85d727a617fc4370e028390ffda68c'
 'ab201e6633e809b642e59df04f1630a5b94ed7c7'
 '0203888f910ce58f835c48e07502837a2a0c7805'
 'd85f230d1b7acaee2594e5fc1b561dd2e4eb3268'
 '79080343587ddb9ec6771c0feb3a05e9c3bb25c3'
 '6bec00f604fade012775a692f45bc784dc6abef8'
 '8c741e27f70785a0add742fad3ee68ec0ea4689a'
 'f9560cc1ed4fd4862babe038346ad2231f610ecc'
 'd4f4c46433e0bac60c61a4c41a9ba01441d0cd68'
 'd36793d64d8d798d3c36003a938e5f3e981f526b'
 'ed72d939f0a045f26ee855cfd968b2207f71f65c'
 'e638f1a5e57beb8bcdb78175dbb73c56cabca5da'
 'cad5236888f7fd9b8f5fc81022d2acc436bab10d'
 '6d7a4742e366fce32df9d48c3adc2d6f882bd154'
 '84ee9f26b9dadebf581410fb711d497be9192041'
 '86fea848310ce46c859586d62431d77bbdd97cfc'
 '37f2ec49cbe558dede84df765d640ee2414b55f0'
 '822ba0794181da3d440e141eba9b5317e0795e01'
 'e33222447d1ec0cb32ff070df682d5a4a61f7856'
 '9ce4322e19f88022c37c1af3e4bd38caf127a2d5'
 '97cd7552a922f510dc7fc4565e3558d2f419d283'
 '04ba30cc74304d7ff219051faa1fac

In [9]:
unique_commits = list(set(unique_commit).symmetric_difference(set(unique_releases_commit)))

In [10]:
unique_commits

['fcb6e3fd6ffff3cda433213b817e8b390596744a',
 '59ac7c6aeecde716326ffd067051720d0417daaa',
 '32dbbdbca780d2ae5da0450cb989a9a7607f89a2',
 '63de01fc754991ac8fa53c78037a118b725be4ce',
 '31ba781997e413bf4e5a57653f47c8b6af6271b3',
 'c508b0e652aa9b9ea4a4dc87d003e22788a361a4',
 'd5656b3ede5b7480936618e4fe428dafe9d1b05f',
 '7289a952f60b21a49d49cbb993972c145c0b6098',
 'a8a55e9df6476dad9ba720ffa63c688da8d3c9bd',
 'b9d8df80f8932aa1c5f1e5efe5071efe0ddd64b8',
 'a010b1486bb0668708cc2b04ddf12c9cb6e835a3',
 'd4f4c46433e0bac60c61a4c41a9ba01441d0cd68',
 '545627aaf16cc13ce72c09e2e2b10d94f7bc8f98',
 '56570505e003b9c9e1ab18f0b686b2ed5735854f',
 '05d58f086f7fd659ee577f598abe18f0fb184bbd',
 'e102036b43100f4b82276013af0414e2008982c4',
 'ebeea85844ba3f6e1dabdec7b0d41517e3839a6f',
 'e3460f86f43e9a78a3c4777029fdf86fbe105903',
 '37f2ec49cbe558dede84df765d640ee2414b55f0',
 '253541f7ef5a698753b17938fbaa4457a4108edb',
 '260a867b79c390dc18c14694af723710595539b9',
 '218b5db7f3c4c11432fd6795242df660fe24fe05',
 '3a59aae8

In [13]:
project_link = "https://github.com/apache/camel"

In [15]:
project_owner = 'apache'
project_name_short = 'camel'

## To obtain total number of pages of commits

In [16]:
owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/repos/{project_owner}/{project_name_short}/commits?per_page=100"

headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers)
print(len(r.json()))

counter = 0
project_name = 'Camel'

with open(f'raw_commits_github_json/{project_name}/{project_name}_{counter}.json', 'w', encoding='utf-8') as f:
    json.dump(r.json(), f, ensure_ascii=False, indent=4)
    
while 'next' in r.links:
    query_url = r.links['next']['url']
    r = requests.get(query_url, headers=headers)
    counter += 1
    with open(f'raw_commits_github_json/{project_name}/{project_name}_{counter}.json', 'a', encoding='utf-8') as f:
        json.dump(r.json(), f, ensure_ascii=False, indent=4)
    
    
f.close()

100


In [17]:
main_arr = []

root_dir = 'raw_commits_github_json/Camel'
data_counter = 0
for filename in os.listdir(root_dir):
    print(filename)

    with open(f'{root_dir}/{filename}', encoding='utf-8') as f:
        data = json.load(f)
        #print(len(data))
        data_counter += len(data)
        
        
        for commits in data:
            try:
                temp_arr = []
                temp_arr.append(commits['sha'])
                temp_arr.append(commits['commit']['author']['name'])
                temp_arr.append(commits['commit']['author']['email'])
                temp_arr.append(commits['commit']['author']['date'])
                temp_arr.append(commits['author']['login'])
                temp_arr.append(commits['commit']['message'])
                temp_arr.append(commits['parents'][0]['sha'])

                main_arr.append(temp_arr)
            except:
                pass
            

Camel_0.json
Camel_1.json
Camel_10.json
Camel_100.json
Camel_101.json
Camel_102.json
Camel_103.json
Camel_104.json
Camel_105.json
Camel_106.json
Camel_107.json
Camel_108.json
Camel_109.json
Camel_11.json
Camel_110.json
Camel_111.json
Camel_112.json
Camel_113.json
Camel_114.json
Camel_115.json
Camel_116.json
Camel_117.json
Camel_118.json
Camel_119.json
Camel_12.json
Camel_120.json
Camel_121.json
Camel_122.json
Camel_123.json
Camel_124.json
Camel_125.json
Camel_126.json
Camel_127.json
Camel_128.json
Camel_129.json
Camel_13.json
Camel_130.json
Camel_131.json
Camel_132.json
Camel_133.json
Camel_134.json
Camel_135.json
Camel_136.json
Camel_137.json
Camel_138.json
Camel_139.json
Camel_14.json
Camel_140.json
Camel_141.json
Camel_142.json
Camel_143.json
Camel_144.json
Camel_145.json
Camel_146.json
Camel_147.json
Camel_148.json
Camel_149.json
Camel_15.json
Camel_150.json
Camel_151.json
Camel_152.json
Camel_153.json
Camel_154.json
Camel_155.json
Camel_156.json
Camel_157.json
Camel_158.json
Camel

In [18]:
data_counter

55105

In [19]:
data[1]['parents'][0]['sha']

'663325c765e47bbd277253e12995e7f22aea9a90'

In [20]:
okhttp_commits_df = pd.DataFrame(main_arr)

In [21]:
okhttp_commits_df.columns = ['sha','name','email','date','login','message', 'parent_sha']

## Creating the dictionary for reference Parent SHA -> Parent SHA Date

In [22]:
parent_sha_dict = {}
for row in okhttp_commits_df.iterrows():
    try:
        # if the commit has a child
        parent_sha_date = okhttp_commits_df[okhttp_commits_df['sha'] == row[1]['parent_sha']]['date'].values[0]
        parent_sha_dict[row[1]['sha']] = parent_sha_date
    except:
        # if the commit does not have a child
        parent_sha_dict[row[1]['sha']] = None
        

In [23]:
parent_sha_dict

{'fe14d164ac23f99c3c5da815a56c1fae7fd6a178': '2021-11-09T10:05:37Z',
 '62147da2c25b72019375acfc469a4413ba18f1c9': '2021-11-09T09:56:37Z',
 '79aab5dcf7154477886b4abbf3e9212b58c91db3': '2021-11-09T09:34:36Z',
 '6998077b5bd7090617d2fbee45c55c5f8d40848c': '2021-11-09T09:21:23Z',
 '7b9a10c7bebc34ee6bde40b667156dd7f4612464': '2021-11-09T08:32:59Z',
 '30494c20e104ffdd2d347966f836d079f5f97d36': '2021-11-08T16:51:22Z',
 'f29fe7e9014f812b301780518ef7c503324d0619': '2021-11-09T07:18:30Z',
 'b73b55a039393621fb5dfa27ffbfcd011060d07c': '2021-11-08T14:42:42Z',
 '464acde20df180c8bbd8b536afb90c9cfdc4dfcd': '2021-11-08T13:45:23Z',
 '868cf9016875075279da7c9616029467db80d019': '2021-11-08T05:28:42Z',
 'fb55460805d6049f4d3529ebf8a7b4a6002f14fa': '2021-11-08T13:10:57Z',
 'be6fe399af530b5290b6c7c2f371df2939471d68': '2021-11-08T11:57:29Z',
 '7933604702897ddf6c7664cfe16eec19983d48e7': '2021-11-08T04:54:39Z',
 'dab936c75d24e565059f08898a0eb451a7f738d1': '2021-11-08T04:53:50Z',
 '50ba19c3758687cd5410a22966172837

In [24]:
def label_parent_date(row):
    return parent_sha_dict[row['sha']]

okhttp_commits_df['parent_date'] = okhttp_commits_df.apply(lambda row: label_parent_date(row), axis=1)

In [25]:
print(len(okhttp_commits_df))

53329


In [26]:
okhttp_commits_df.dtypes

sha            object
name           object
email          object
date           object
login          object
message        object
parent_sha     object
parent_date    object
dtype: object

In [27]:
okhttp_commits_df['date'] = pd.to_datetime(okhttp_commits_df['date'], infer_datetime_format=True)
okhttp_commits_df['parent_date'] = pd.to_datetime(okhttp_commits_df['parent_date'], infer_datetime_format=True)

In [28]:
okhttp_commits_df['time_taken'] = (okhttp_commits_df['date'] - okhttp_commits_df['parent_date']).astype('timedelta64[h]')

In [29]:
okhttp_commits_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken
0,fe14d164ac23f99c3c5da815a56c1fae7fd6a178,Otavio Rodolfo Piske,opiske@redhat.com,2021-11-09 10:06:31,orpiske,CAMEL-17121: converted camel-splunk to repeata...,62147da2c25b72019375acfc469a4413ba18f1c9,2021-11-09 10:05:37,0.0
1,62147da2c25b72019375acfc469a4413ba18f1c9,aldettinger,aldettinger@gmail.com,2021-11-09 10:05:37,aldettinger,CAMEL-11178: Fix wrong HTTP link to Woodstox,79aab5dcf7154477886b4abbf3e9212b58c91db3,2021-11-09 09:56:37,0.0
2,79aab5dcf7154477886b4abbf3e9212b58c91db3,aldettinger,aldettinger@gmail.com,2021-11-09 09:56:37,aldettinger,CAMEL-17178: Fix wrong HTTP links to Jackson J...,6998077b5bd7090617d2fbee45c55c5f8d40848c,2021-11-09 09:34:36,0.0
3,6998077b5bd7090617d2fbee45c55c5f8d40848c,aldettinger,aldettinger@gmail.com,2021-11-09 09:34:36,aldettinger,CAMEL-17178: Fix wrong HTTP links to Jackson l...,7b9a10c7bebc34ee6bde40b667156dd7f4612464,2021-11-09 09:21:23,0.0
4,7b9a10c7bebc34ee6bde40b667156dd7f4612464,aldettinger,aldettinger@gmail.com,2021-11-09 09:21:23,aldettinger,CAMEL-17178: Remove the useless reference to w...,30494c20e104ffdd2d347966f836d079f5f97d36,2021-11-09 08:32:59,0.0


In [30]:
def check_if_refactoring_present_in_commit(row):
    if row['sha'] in unique_commits:
        return 1
    return 0

In [31]:
okhttp_commits_df['contains_refactoring'] = okhttp_commits_df.apply(lambda row: check_if_refactoring_present_in_commit(row), axis=1)

In [32]:
okhttp_commits_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring
0,fe14d164ac23f99c3c5da815a56c1fae7fd6a178,Otavio Rodolfo Piske,opiske@redhat.com,2021-11-09 10:06:31,orpiske,CAMEL-17121: converted camel-splunk to repeata...,62147da2c25b72019375acfc469a4413ba18f1c9,2021-11-09 10:05:37,0.0,0
1,62147da2c25b72019375acfc469a4413ba18f1c9,aldettinger,aldettinger@gmail.com,2021-11-09 10:05:37,aldettinger,CAMEL-11178: Fix wrong HTTP link to Woodstox,79aab5dcf7154477886b4abbf3e9212b58c91db3,2021-11-09 09:56:37,0.0,0
2,79aab5dcf7154477886b4abbf3e9212b58c91db3,aldettinger,aldettinger@gmail.com,2021-11-09 09:56:37,aldettinger,CAMEL-17178: Fix wrong HTTP links to Jackson J...,6998077b5bd7090617d2fbee45c55c5f8d40848c,2021-11-09 09:34:36,0.0,0
3,6998077b5bd7090617d2fbee45c55c5f8d40848c,aldettinger,aldettinger@gmail.com,2021-11-09 09:34:36,aldettinger,CAMEL-17178: Fix wrong HTTP links to Jackson l...,7b9a10c7bebc34ee6bde40b667156dd7f4612464,2021-11-09 09:21:23,0.0,0
4,7b9a10c7bebc34ee6bde40b667156dd7f4612464,aldettinger,aldettinger@gmail.com,2021-11-09 09:21:23,aldettinger,CAMEL-17178: Remove the useless reference to w...,30494c20e104ffdd2d347966f836d079f5f97d36,2021-11-09 08:32:59,0.0,0


In [33]:
okhttp_commits_df['contains_refactoring'].value_counts()

0    53093
1      236
Name: contains_refactoring, dtype: int64

In [34]:
okhttp_commits_df.to_csv('raw_commits_github_json/Camel/Camel_commits.csv',index=False)

In [2]:
okhttp_commits_df = pd.read_csv('raw_commits_github_json/Okhttp/Okhttp_commits.csv')

In [3]:
okhttp_commits_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring
0,3ef564b0ddcb00655e540e52ad882a5881c1dc54,Jesse Wilson,jesse@swank.ca,2021-11-01 13:29:31,swankjesse,Don't document mockwebserver3.internal (#6895),1ed9863131c87420381d8a6be48b96157a9bb764,2021-10-29 07:39:56,77.0,0
1,1ed9863131c87420381d8a6be48b96157a9bb764,Jesse Wilson,jesse@swank.ca,2021-10-29 07:39:56,swankjesse,Upgrade to Okio 3.0.0 (#6896),bcfdb4f2ebd9d5ccfb1452568073e4f2116c6738,2021-10-22 16:51:49,158.0,0
2,bcfdb4f2ebd9d5ccfb1452568073e4f2116c6738,Jim Hurne,hurne_jim@yahoo.com,2021-10-22 16:51:49,jthurne,Fix EventSources.processResponse to not throw ...,1ff16232f86d2b302840a3bb2f9d386e94374753,2021-10-15 05:04:08,179.0,0
3,1ff16232f86d2b302840a3bb2f9d386e94374753,Clément Dessoude,clement.dessoude@gmail.com,2021-10-15 05:04:08,clementdessoude,fix(docs): update broken link on docs homepage...,a87f4b4585fe6e8e49c7760986fd5d2c591e0573,2021-10-01 01:13:35,339.0,0
4,a87f4b4585fe6e8e49c7760986fd5d2c591e0573,Jesse Wilson,jwilson@squareup.com,2021-10-01 01:13:35,swankjesse,Changelog for OkHttp 4.9.2,286187102848b30d9580794795746b28462a79d0,2021-09-24 06:13:58,162.0,0


In [35]:
query_url = f"https://api.github.com/rate_limit"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers)
pprint(r.json())
print(len(r.json()))

https://api.github.com/rate_limit
{'rate': {'limit': 5000, 'remaining': 4447, 'reset': 1636467068, 'used': 553},
 'resources': {'actions_runner_registration': {'limit': 10000,
                                               'remaining': 10000,
                                               'reset': 1636467799,
                                               'used': 0},
               'code_scanning_upload': {'limit': 500,
                                        'remaining': 500,
                                        'reset': 1636467799,
                                        'used': 0},
               'core': {'limit': 5000,
                        'remaining': 4447,
                        'reset': 1636467068,
                        'used': 553},
               'graphql': {'limit': 5000,
                           'remaining': 5000,
                           'reset': 1636467799,
                           'used': 0},
               'integration_manifest': {'limit': 5000,
          